In [1]:
import os
import time
import argparse

import torchvision
import torch
import torch.nn as nn

from util import AverageMeter
from encoder import SmallAlexNet
from align_uniform import align_loss, uniform_loss_prelog
from tqdm import tqdm
from collections import defaultdict
import copy

import matplotlib.pyplot as plt


class TwoAugUnsupervisedDatasetLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, transform, lblmap=None):
        self.dataset = dataset
        self.transform = transform
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        lbl2return = lbl if self.lblmap is None else self.lblmap[lbl]
        return self.transform(image), self.transform(image), lbl2return

    def __len__(self):
        return len(self.dataset)

def parse_option():
    parser = argparse.ArgumentParser('STL-10 Representation Learning with Alignment and Uniformity Losses')

    parser.add_argument('--align_w', type=float, default=1, help='Alignment loss weight')
    parser.add_argument('--unif_w', type=float, default=1, help='Uniformity loss weight')
    parser.add_argument('--align_alpha', type=float, default=2, help='alpha in alignment loss')
    parser.add_argument('--unif_t', type=float, default=2, help='t in uniformity loss')

    parser.add_argument('--batch_size', type=int, default=256, help='Batch size')
    parser.add_argument('--epochs', type=int, default=400, help='Number of training epochs')
    parser.add_argument('--iter', type=int, default=0, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=None,
                        help='Learning rate. Default is linear scaling 0.12 per 256 batch size')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1, help='Learning rate decay rate')
    parser.add_argument('--lr_decay_epochs', default=[155, 170, 185], nargs='*', type=int,
                        help='When to decay learning rate')
    parser.add_argument('--momentum', type=float, default=0.9, help='SGD momentum')
    parser.add_argument('--weight_decay', type=float, default=1e-4, help='L2 weight decay')
    parser.add_argument('--feat_dim', type=int, default=128, help='Feature dimensionality')

    parser.add_argument('--num_workers', type=int, default=4, help='Number of data loader workers to use')
    parser.add_argument('--log_interval', type=int, default=40, help='Number of iterations between logs')
    parser.add_argument('--gpus', default=[0], nargs='*', type=int,
                        help='List of GPU indices to use, e.g., --gpus 0 1 2 3')

    parser.add_argument('--data_folder', type=str, default='./data', help='Path to data')
    parser.add_argument('--result_folder', type=str, default='./results', help='Base directory to save model')

    opt = parser.parse_args("")

    if opt.lr is None:
        opt.lr = 0.12 * (opt.batch_size / 256)

    opt.gpus = list(map(lambda x: torch.device('cuda', x), opt.gpus))

    opt.save_folder = os.path.join(
        opt.result_folder,
        f"cifar100_400_sideinformation_align{opt.align_w:g}alpha{opt.align_alpha:g}_unif{opt.unif_w:g}t{opt.unif_t:g}_iter{opt.iter}"
    )
    os.makedirs(opt.save_folder, exist_ok=True)

    return opt


opt = parse_option()

In [2]:
transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        torchvision.transforms.RandomGrayscale(p=0.2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])

old_lbls = list(range(100))
labels_2_keep = list(range(25))
# labels_2_keep = [0,1,2,3]

old2new = {}
count = 0
for old_lbl in old_lbls:
    if old_lbl in labels_2_keep: 
        old2new[old_lbl] = count
        count += 1

for old_lbl in old_lbls:
    if old_lbl not in labels_2_keep: 
        old2new[old_lbl] = count

new_lbls = list(range(count+1))

In [ ]:
def get_data_loader(opt):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
        torchvision.transforms.RandomGrayscale(p=0.2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    dataset = TwoAugUnsupervisedDatasetLbl(
        torchvision.datasets.CIFAR100(opt.data_folder, 'train', download=True), 
        transform=transform, 
        lblmap=old2new )
    
    return torch.utils.data.DataLoader(dataset, batch_size=opt.batch_size, num_workers=opt.num_workers,
                                       shuffle=True, pin_memory=True)


print(f'Optimize: {opt.align_w:g} * loss_align(alpha={opt.align_alpha:g}) + {opt.unif_w:g} * loss_uniform(t={opt.unif_t:g})')

torch.cuda.set_device(opt.gpus[0])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

encoder = SmallAlexNet(feat_dim=opt.feat_dim, cifar=True).to(opt.gpus[0])

optim = torch.optim.Adam(encoder.parameters(), lr=1e-2)

loader = get_data_loader(opt)
align_meter = AverageMeter('align_loss')
unif_meter = AverageMeter('uniform_loss')
loss_meter = AverageMeter('total_loss')
it_time_meter = AverageMeter('iter_time')

for epoch in range(opt.epochs):
    align_meter.reset()
    unif_meter.reset()
    loss_meter.reset()
    it_time_meter.reset()
    t0 = time.time()
    for ii, (im_x, im_y, lbl) in enumerate(loader):
        optim.zero_grad()
        x, y = encoder(torch.cat([im_x.to(opt.gpus[0]), im_y.to(opt.gpus[0])])).chunk(2)
        
        align_loss_val = align_loss(x, y, alpha=opt.align_alpha)
        # group according to new_lbls

        z = torch.cat( [x, y])
        lbl_z = torch.cat([lbl, lbl])
        unif_losses = torch.cat([uniform_loss_prelog(z[lbl_z==new_lbl]) for new_lbl in new_lbls])
        unif_loss_val = torch.log( torch.mean(unif_losses) )
        
        loss = align_loss_val * opt.align_w + unif_loss_val * opt.unif_w
        align_meter.update(align_loss_val, x.shape[0])
        unif_meter.update(unif_loss_val)
        loss_meter.update(loss, x.shape[0])
        loss.backward()
        optim.step()
        it_time_meter.update(time.time() - t0)
        if ii % opt.log_interval == 0:
            print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(loader)}\t" +
                  f"{align_meter}\t{unif_meter}\t{loss_meter}\t{it_time_meter}")
        t0 = time.time()

ckpt_file = os.path.join(opt.save_folder, 'encoder.pth')
torch.save(encoder.state_dict(), ckpt_file)
print(f'Saved to {ckpt_file}')

Optimize: 1 * loss_align(alpha=2) + 1 * loss_uniform(t=2)
Files already downloaded and verified
Epoch 0/400	It 0/196	align_loss 1.170543 (1.170543)	uniform_loss -2.484235 (-2.484235)	total_loss -1.313692 (-1.313692)	iter_time 3.413382 (3.413382)


In [ ]:
"""
    Here we  do the linear evaluation, the old labels are provided to the linear objective as one hot
"""
import time
import argparse

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

from util import AverageMeter
from encoder import SmallAlexNet
MODEL_2_LOAD = "./results/base_200_sideinformation_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD = "./results/base_200_4_sideinformation_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/manual_labels_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/cifar100_manual_labels_align1alpha2_unif1t2_iter0/encoder.pth"
MODEL_2_LOAD="./results/cifar100_sideinformation_align1alpha2_unif1t2_iter0/encoder.pth"

USE_MOD_LBL = False
encoder = SmallAlexNet(feat_dim=opt.feat_dim, cifar=True).to(opt.gpus[0])
encoder.load_state_dict(torch.load(MODEL_2_LOAD))

In [10]:
def parse_option():
    parser = argparse.ArgumentParser('STL-10 Representation Learning with Alignment and Uniformity Losses')

    parser.add_argument('--encoder_checkpoint', type=str, help='Encoder checkpoint to evaluate', default=MODEL_2_LOAD)
    parser.add_argument('--feat_dim', type=int, default=128, help='Encoder feature dimensionality')
    parser.add_argument('--layer_index', type=int, default=-2, help='Evaluation layer')

    parser.add_argument('--batch_size', type=int, default=128, help='Batch size')
    parser.add_argument('--epochs', type=int, default=100, help='Number of training epochs')
    parser.add_argument('--lr', type=float, default=1e-3, help='Learning rate')
    parser.add_argument('--lr_decay_rate', type=float, default=0.2, help='Learning rate decay rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='60,80', help='When to decay learning rate')

    parser.add_argument('--num_workers', type=int, default=6, help='Number of data loader workers to use')
    parser.add_argument('--log_interval', type=int, default=40, help='Number of iterations between logs')
    parser.add_argument('--gpus', default=[0], nargs='*', type=int,
                        help='List of GPU indices to use, e.g., --gpus 0 1 2 3')

    parser.add_argument('--data_folder', type=str, default='./data', help='Path to data')

    opt = parser.parse_args("")

    if opt.lr is None:
        opt.lr = 0.12 * (opt.batch_size / 256)

    opt.gpu = torch.device('cuda', opt.gpus[0])
    opt.lr_decay_epochs = list(map(int, opt.lr_decay_epochs.split(',')))

    return opt


class DatasetModifiedLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap=None):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        lbl2return = lbl if self.lblmap is None else self.lblmap[lbl]
        return image, lbl2return

    def __len__(self):
        return len(self.dataset)

In [11]:
class DatasetModifiedLblandLbl(torch.utils.data.Dataset):
    r"""Returns two augmentation and no labels."""

    def __init__(self, dataset, lblmap):
        self.dataset = dataset
        self.lblmap = copy.deepcopy(lblmap)

    def __getitem__(self, index):
        image, lbl = self.dataset[index]
        return image, self.lblmap[lbl], lbl

    def __len__(self):
        return len(self.dataset)

In [12]:
def get_data_loaders(opt):
    train_transform = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(32, scale=(0.08, 1)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
        torchvision.transforms.RandomHorizontalFlip()
    ])
    val_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(70),
        torchvision.transforms.CenterCrop(32),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
            (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
        ),
    ])
    train_dataset = DatasetModifiedLblandLbl( torchvision.datasets.CIFAR100(opt.data_folder, 'train', download=True, transform=train_transform), lblmap=old2new)
    val_dataset =  DatasetModifiedLblandLbl( torchvision.datasets.CIFAR100(opt.data_folder, 'test', transform=val_transform), lblmap=old2new)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size,
                                               num_workers=opt.num_workers, shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=opt.batch_size,
                                             num_workers=opt.num_workers, pin_memory=True)
    return train_loader, val_loader


def validate_comb(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_mod, labels_act in val_loader:
            pred = classifier(torch.cat( (encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1), torch.nn.functional.one_hot(labels_mod.to(opt.gpus[0]), num_classes=len(labels_2_keep)+1)), dim=1)).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

def validate(opt, encoder, classifier, val_loader):
    correct = 0
    with torch.no_grad():
        for images, labels_mod, labels_act in val_loader:
            pred = classifier( encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1) ).argmax(dim=1)
            correct += (pred.cpu() == labels_act).sum().item()
    return correct / len(val_loader.dataset)

In [13]:
opt.gpu=opt.gpus[0]

In [14]:
opt = parse_option()

opt.gpu=opt.gpus[0]
torch.cuda.set_device(opt.gpus[0])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

encoder.eval()
train_loader, val_loader = get_data_loaders(opt)

with torch.no_grad():
    sample, _ = train_loader.dataset.dataset[0]
    eval_numel = encoder(sample.unsqueeze(0).to(opt.gpus[0]), layer_index=opt.layer_index).numel()
print(f'Feature dimension: {eval_numel}')


classifier = nn.Linear(eval_numel, 100).to(opt.gpus[0]) if not USE_MOD_LBL else nn.Linear( eval_numel + len(labels_2_keep) + 1,100).to(opt.gpus[0])

optim = torch.optim.Adam(classifier.parameters(), lr=opt.lr, betas=(0.5, 0.999))
val_accs = []
loss_meter = AverageMeter('loss')
it_time_meter = AverageMeter('iter_time')
for epoch in tqdm(range(opt.epochs)):
    loss_meter.reset()
    it_time_meter.reset()
    t0 = time.time()
    for ii, (images, labels_mod, labels) in enumerate(train_loader):
        optim.zero_grad()
        with torch.no_grad():
            feats = encoder(images.to(opt.gpus[0]), layer_index=opt.layer_index).flatten(1)

        if USE_MOD_LBL:
            logits = classifier(torch.cat( (feats, torch.nn.functional.one_hot(labels_mod.to(opt.gpus[0]), num_classes=len(labels_2_keep)+1 )),dim=1))
        else:
            logits = classifier(feats)
        
        loss = F.cross_entropy(logits, labels.to(opt.gpus[0]))
        loss_meter.update(loss, images.shape[0])
        loss.backward()
        optim.step()
        it_time_meter.update(time.time() - t0)
        if ii % opt.log_interval == 0:
            print(f"Epoch {epoch}/{opt.epochs}\tIt {ii}/{len(train_loader)}\t{loss_meter}\t{it_time_meter}")
        t0 = time.time()
    val_acc = validate_comb(opt,encoder,classifier,val_loader) if USE_MOD_LBL else validate(opt, encoder, classifier, val_loader) 
    val_accs.append(val_acc)
    print(f"Epoch {epoch}/{opt.epochs}\tval_acc {val_acc*100:.4g}%")
print(f"Best validation accuracy {max(val_accs)}")

Files already downloaded and verified
Feature dimension: 4096


  0%|                                                                                                                                                  | 0/100 [00:00<?, ?it/s]

Epoch 0/100	It 0/391	loss 4.607521 (4.607521)	iter_time 0.099628 (0.099628)
Epoch 0/100	It 40/391	loss 3.905118 (4.197085)	iter_time 0.005078 (0.010257)
Epoch 0/100	It 80/391	loss 3.484926 (3.905156)	iter_time 0.005043 (0.009125)
Epoch 0/100	It 120/391	loss 3.272768 (3.715056)	iter_time 0.005205 (0.008926)
Epoch 0/100	It 160/391	loss 3.061009 (3.562499)	iter_time 0.005024 (0.008908)
Epoch 0/100	It 200/391	loss 2.903367 (3.454365)	iter_time 0.004706 (0.008753)
Epoch 0/100	It 240/391	loss 2.838861 (3.358420)	iter_time 0.009608 (0.008611)
Epoch 0/100	It 280/391	loss 2.703639 (3.285984)	iter_time 0.030228 (0.008636)
Epoch 0/100	It 320/391	loss 2.756047 (3.223621)	iter_time 0.029022 (0.008637)
Epoch 0/100	It 360/391	loss 2.799281 (3.171773)	iter_time 0.004683 (0.008645)


  1%|█▍                                                                                                                                        | 1/100 [00:06<10:16,  6.23s/it]

Epoch 0/100	val_acc 32.84%
Epoch 1/100	It 0/391	loss 2.739886 (2.739886)	iter_time 0.103920 (0.103920)
Epoch 1/100	It 40/391	loss 2.705095 (2.662412)	iter_time 0.004777 (0.011043)
Epoch 1/100	It 80/391	loss 2.461795 (2.632401)	iter_time 0.018184 (0.010019)
Epoch 1/100	It 120/391	loss 2.613197 (2.632971)	iter_time 0.004960 (0.009194)
Epoch 1/100	It 160/391	loss 2.644512 (2.631217)	iter_time 0.019723 (0.008979)
Epoch 1/100	It 200/391	loss 2.579678 (2.625735)	iter_time 0.005044 (0.008762)
Epoch 1/100	It 240/391	loss 2.530645 (2.615306)	iter_time 0.004725 (0.008609)
Epoch 1/100	It 280/391	loss 2.685738 (2.610246)	iter_time 0.016864 (0.008615)
Epoch 1/100	It 320/391	loss 2.395944 (2.601833)	iter_time 0.004920 (0.008676)
Epoch 1/100	It 360/391	loss 2.483490 (2.593659)	iter_time 0.004784 (0.008666)


...

Epoch 97/100	val_acc 47.84%
Epoch 98/100	It 0/391	loss 1.692616 (1.692616)	iter_time 0.100038 (0.100038)
Epoch 98/100	It 40/391	loss 2.076674 (1.748595)	iter_time 0.005013 (0.010117)
Epoch 98/100	It 80/391	loss 1.790367 (1.760427)	iter_time 0.005008 (0.009295)
Epoch 98/100	It 120/391	loss 1.577076 (1.750340)	iter_time 0.005146 (0.008710)
Epoch 98/100	It 160/391	loss 1.778699 (1.739570)	iter_time 0.005056 (0.008423)
Epoch 98/100	It 200/391	loss 1.690460 (1.741975)	iter_time 0.004978 (0.008404)
Epoch 98/100	It 240/391	loss 1.739845 (1.736997)	iter_time 0.004817 (0.008309)
Epoch 98/100	It 280/391	loss 1.721685 (1.739955)	iter_time 0.004850 (0.008433)
Epoch 98/100	It 320/391	loss 1.666591 (1.744621)	iter_time 0.004955 (0.008484)
Epoch 98/100	It 360/391	loss 1.931059 (1.744200)	iter_time 0.032913 (0.008608)


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 99/100 [10:05<00:06,  6.08s/it]

Epoch 98/100	val_acc 47.84%
Epoch 99/100	It 0/391	loss 1.605502 (1.605502)	iter_time 0.118343 (0.118343)
Epoch 99/100	It 40/391	loss 1.685499 (1.738668)	iter_time 0.005188 (0.011644)
Epoch 99/100	It 80/391	loss 1.598729 (1.760959)	iter_time 0.004999 (0.010268)
Epoch 99/100	It 120/391	loss 1.553457 (1.752099)	iter_time 0.020096 (0.009532)
Epoch 99/100	It 160/391	loss 1.728272 (1.758151)	iter_time 0.004851 (0.009234)
Epoch 99/100	It 200/391	loss 1.560333 (1.753113)	iter_time 0.004815 (0.008991)
Epoch 99/100	It 240/391	loss 1.898610 (1.750765)	iter_time 0.004934 (0.008867)
Epoch 99/100	It 280/391	loss 1.786513 (1.746032)	iter_time 0.004962 (0.008789)
Epoch 99/100	It 320/391	loss 1.479509 (1.745728)	iter_time 0.004622 (0.008773)
Epoch 99/100	It 360/391	loss 1.597241 (1.748891)	iter_time 0.004755 (0.008712)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:11<00:00,  6.11s/it]

Epoch 99/100	val_acc 47.83%
Best validation accuracy 0.47864
